# Create assistant with access to files and a function

This assistant can answer questions about the HR policy of the fictional company Innovatek. It has access to the following files:

- Innovatek.pdf

There is also a function that allows an employee to request a raise.

**Note**: in the OpenAI Assistants API (not the Azure one), you can upload files and enable the **retrieval** tool like so:

```python
assistant = client.beta.assistants.create(
  instructions="You are a customer support chatbot. Use your knowledge base to best respond to customer queries.",
  model="gpt-4-turbo-preview",
  tools=[{"type": "retrieval"}]
)
```

You would also upload files and pass the file ids here but that is not shown.

Currently, the Azure OpenAI Assistants API does not allow you to configure the retrieval tool. You can only enable the `code_interpreter` tool. That tool might be able to open your documents and read them, but it's not as powerful as the retrieval tool. It basically has to use Python code to read the file and then answer questions about it.

In [6]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

# Load environment variables from .env file
# AZURE_OPENAI_API_KEY
# AZURE_OPENAI_ENDPOINT
# AZURE_OPENAI_API_VERSION
load_dotenv()

# Create Azure OpenAI client
client = AzureOpenAI(
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    api_version=os.getenv('AZURE_OPENAI_API_VERSION')
)

# upload the HR policy and get the file id
hr_policy_file = client.files.create(
    file=open('Innovatek.pdf', 'rb'),
    purpose='assistants'
)

# load assistant ID from assistant_id.txt
try:
    with open('assistant_id.txt', 'r') as f:
        assistant_id = f.read()
except FileNotFoundError:
    print("The file 'assistant_id.txt' does not exist.")
    assistant_id = None

print('Assistant ID:', assistant_id)

if assistant_id is None:
    # Create a new assistant
    assistant = client.beta.assistants.create(
        name="HR Assistant",
        instructions="""You are an HR Assistant for the company Innovatek.
        You answer questions about HR policy from the files you have access to. The policy is in Innovatek.pdf.
        Users can ask for a raise. If they do, so call the function `request_raise` with the amount they are asking for.
        """,
        tools=[{ 
            "function": {
                "name": "request_raise",
                "description": "Request a raise for an employee",
                "parameters": {
                "type": "object",
                "properties": {
                    "employee": {
                        "type": "string",
                        "description": "Name of the employee. Ask the user if you do not know."
                        },
                    "amount": {
                        "type": "integer",
                        "description": "The raise amount, not the new salary"
                        }
                },
                "required": [
                    "employee",
                    "amount"
                ]
                }
            },
            "type": "function"
            }, 
            {
                "type": "code_interpreter",  # should be set to retrieval but that is not supported yet
            }
        ],
        model="gpt-4-preview", # ensure you have a deployment in the region you are using
        file_ids=[hr_policy_file.id] # pass the file
    )

    
    assistant_id = assistant.id
    with open('assistant_id.txt', 'w') as f:
        f.write(assistant_id)
    print('Assistant created:', assistant_id)
    

The file 'assistant_id.txt' does not exist.
Assistant ID: None
Assistant created: asst_EXRAVxcQH4ZN9PxEQ310KbER


## Create a thread and add a message

In [7]:
# Create a thread
thread = client.beta.threads.create()

# Threads have an id as well
print("Thread id: ", thread.id)

import time
from IPython.display import clear_output

# function returns the run when status is no longer queued or in_progress
def wait_for_run(run, thread_id):
    while run.status == 'queued' or run.status == 'in_progress':
        run = client.beta.threads.runs.retrieve(
                thread_id=thread_id,
                run_id=run.id
        )
        time.sleep(0.5)

    return run


# create a message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can I wear short pants to work?"
)

# create a run 
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id, # use the assistant id defined in the first cell
    # tools=[{"type": "retrieval"}] # using the retrieval tool is not supported at run level either
)

# wait for the run to complete
run = wait_for_run(run, thread.id)

# show information about the run
# should indicate that run status is requires_action
# should contain information about the tools to call
print(run.model_dump_json(indent=2))

Thread id:  thread_LYT23JlsAiPVkrQsQWCeiJAi


BadRequestError: Error code: 400 - {'error': {'message': 'Retrieval tools are not supported.', 'type': 'invalid_request_error', 'param': 'tools[0]', 'code': None}}